In [3]:
#imports
import re
import pandas as pd
from textblob import TextBlob

#pulls data from file
data_path = 'C:/Users/kingr/Desktop/sentiment analysis/Chat.txt'
data=open(data_path, 'r', encoding='utf-8')
lines=data.readlines()
data.close()


#splits data into date, time, message
def split_data(line):
    pattern = r'^(\d{2}/\d{2}/\d{2}), (\d{1,2}:\d{2})\s?(am|pm) - (\+?\d{3} \d{2} \d{3} \d{4}): (.+)$'
    neat_data=[]

    match=re.match(pattern, line)

    if match:
        date= match.group(1)
        time= match.group(2)+" "+match.group(3)
        user= match.group(4)
        message= match.group(5)
        return date, time, user, message

    else:
        return None

date_data=[]
time_data=[]
users_data=[]
message_data=[]

#process each line
for line in lines:
    detail=split_data(line)
    
    if detail:
        date, time, user, message=detail
        date_data.append(date)
        time_data.append(time)
        users_data.append(user)
        message_data.append(message)
        

sentiment_score=[]
sentiment_value=[]
for messages in message_data:
    check=TextBlob(messages)
    sentiment=check.sentiment.polarity
    sentiment_score.append(sentiment)
    
    if sentiment > 0:
        sentiment_value.append("Positive")
    elif sentiment == 0:
        sentiment_value.append("Neutral")
    else:
        sentiment_value.append("Negative")

        
pd.set_option('display.max_columns', 100000000)
pd.set_option('display.width', 10000000)
pd.set_option('display.precision', 2)
pd.set_option('display.colheader_justify', 'center')
#pd.set_option('display.notebook_repr_html', True)

        
table_data = pd.DataFrame({"DATE": date_data,
                           "TIME": time_data,
                           "USERS": users_data,
                           "MESSAGES": message_data,
                           "SENTIMENT SCORE": sentiment_score,
                           "SENTIMENT VALUE": sentiment_value})

table_data.to_csv('Final data.csv', index=False)
print(table_data)

         DATE      TIME         USERS                            MESSAGES                       SENTIMENT SCORE SENTIMENT VALUE
0      23/09/23   2:06 pm  +233 55 624 6989  *CANADA 🇨🇦 FULLY FUNDED SCHOLARSHIPS 2023/2024...       0.00            Neutral   
1      24/09/23   8:39 pm  +233 59 273 8323                                       ‼️‼️‼️‼️‼️‼️       0.00            Neutral   
2      24/09/23   8:39 pm  +233 55 427 0384                                            Chai 😂😂       0.00            Neutral   
3      24/09/23   8:40 pm  +233 55 631 5930                                           Skkrrr 😂       0.00            Neutral   
4      24/09/23   8:44 pm  +233 55 052 9435                  So people really check this errr😂       0.20           Positive   
...         ...       ...               ...                                                ...              ...             ...
11023  21/04/24   7:33 pm  +233 50 561 5705                                           Changed.       0.0